<a href="https://colab.research.google.com/github/OluwoleDove/ML_class/blob/main/Inniyet_phase_one_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install fuzzywuzzy

In [ ]:
pip install squarify

## **IMPORT MODULES**

In [1]:
import numpy as np
import pandas as pd
import re
from fuzzywuzzy import process
import csv
import squarify

/usr/local/lib/python3.10/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


## **READ FILES**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df2 = pd.read_excel("/content/drive/MyDrive/inniyet_data/MSc Project/DISCOVERED OESTROGENS.xlsx")
df3 = pd.read_csv("/content/drive/MyDrive/inniyet_data/MSc Project/sec_data.csv")

NameError: name 'pd' is not defined

In [ ]:
df2.head()

,BNF chemical name,Drug/device name,Drug FORM,Drug strength,Item description,BNF sub-section,BNF sub-section.1,Number of dispensed items,Dispensed quantity,Gross ingredient cost (£),Cost per item (£),Quantity per item
0,DROSPIRENONE WITH ETHINYLESTRADIOL,ACONDRO,TABLET,3MG/30MCG,NaN,COMBINED HORMONAL CONTRACEPTIVES AND SYS,70301,7,567.0,75.15,10.735714,81.000000
1,DROSPIRENONE WITH ETHINYLESTRADIOL,DRETINE,TABLET,3MG/30MCG,NaN,COMBINED HORMONAL CONTRACEPTIVES AND SYS,70301,823,87488.0,11487.23,13.957752,106.303767
2,DROSPIRENONE WITH ETHINYLESTRADIOL,YASMIN,TABLET,3MG/30MCG,NaN,COMBINED HORMONAL CONTRACEPTIVES AND SYS,70301,3755,366767.0,84986.02,22.632762,97.674301
3,ESTRADIOL,ELLESTE SOLO,TABLET,1 MG,NaN,FEMALE SEX HORMONES AND THEIR MODULATORS,60401,14288,1253580.0,75102.40,5.256327,87.736562
4,ESTRADIOL VALERATE,PROGYNOVA,TABLET,2 MG,NaN,FEMALE SEX HORMONES AND THEIR MODULATORS,60401,4923,495971.0,42809.95,8.695907,100.745684


In [ ]:
df3.head(5)

,Product Name,Composition,Pharmaceutical Form,Clinical Particulars
0,Cetraben Emollient Bath Additive Cetraben Bat...,Light Liquid Paraffin 82.8% w/w For excipients...,Bath additive Cetraben Emollient Bath Additive...,NaN
1,"Promixin, 1 million International Units (IU) P...",Each vial contains 1 million International Uni...,Powder for nebuliser solution. The powder is w...,NaN
2,Duac Once Daily 10 mg/g + 50 mg/g Gel,1 g of gel contains: 10 mg clindamycin as clin...,Gel White to slightly yellow homogeneous gel,NaN
3,Prosulf® 10mg/ml Solution for Injection Prota...,Protamine Sulfate 10mg/ml,"Solution for injection A clear, colourless so...",NaN
4,Tensipine MR 10Tensipine MR 20,Tensipine MR 10 One film-coated tablet contai...,Modified release tablets for oral administrati...,NaN


## **PRELIMINARY DATA CLEANING**

In [ ]:
df2.columns.tolist()

['BNF chemical name',
 'Drug/device name',
 'Drug FORM',
 'Drug strength',
 'Item description',
 'BNF sub-section',
 'BNF sub-section.1',
 'Number of dispensed items',
 'Dispensed quantity',
 'Gross ingredient cost (£)',
 'Cost per item (£)',
 'Quantity per item']

In [ ]:
df3.columns.tolist()

['Product Name', 'Composition', 'Pharmaceutical Form', 'Clinical Particulars']

## **FILLING BLANKS IN SECONDARY DATA**

In [ ]:
'''Estradiol/dinogest has drug strength 3mg/500mcG
Estradiol is 3mg | Dinogest is 500mcg which is 0.5mg'''
cleaned_data_rows = []

# Iterate over each row in df2
for index, row in df2.iterrows():
    drug_name = row["Drug/device name"]

    # Use fuzzy matching to find the closest match in df3's Product Name
    matching_name = process.extractOne(drug_name, df3["Product Name"])

    # Check if a matching name was found
    if matching_name:
        # Get the matching name
        matching_name = matching_name[0]

        # Find the corresponding row in df3
        matching_row = df3[df3["Product Name"] == matching_name].iloc[0]

        # Extract relevant information
        drug_form = matching_row["Pharmaceutical Form"]

        # Extract drug strength and item description from composition
        composition = matching_row["Composition"]
        # Extract drug strength from composition
        drug_strength_match = re.search(r'\d*\.?\d+\s*(mg|mcg|ml|g|micrograms)', composition, re.IGNORECASE)
        drug_strength = drug_strength_match.group() if drug_strength_match else ""
        # Remove drug strength from composition
        composition = re.sub(drug_strength, "", composition, flags=re.IGNORECASE).strip()
        # Summarize drug form
        drug_form = drug_form.split('.')[0].strip()

        # Append the extracted information to the list of cleaned data rows
        cleaned_data_rows.append({
            "Drug/device name": drug_name,
            "Drug FORM": drug_form,
            "Drug strength": drug_strength,
            "Item description": composition
        })

# Create a DataFrame from the list of cleaned data rows
cleaned_data = pd.DataFrame(cleaned_data_rows)

# Write the cleaned data to a CSV file
cleaned_data.to_csv("secondary_df.csv", index=False)

## **REDRESSING DATA FILES**

In [ ]:
df4 = pd.read_csv("/content/drive/MyDrive/inniyet_data/MSc Project/secondary_df.csv")

columns_to_fill = ['Drug FORM', 'Drug strength', 'Item description']

# Iterate over each column to fill
for column in columns_to_fill:
    # Find indices of rows with missing values in the current column
    missing_indices = df2[df2[column].isna()].index

    # Iterate over missing indices
    for index in missing_indices:
        # Find corresponding value in df4
        value_to_fill = df4.at[index, column]
        df2.at[index, column] = value_to_fill


df2.to_excel('DISCOVERED_OESTROGENS_MODIFIED.xlsx', index=False)